In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# ————————————————————————————————
# 1A) Define Generator & Discriminator
# ————————————————————————————————

latent_dim = 100
num_classes = 3
img_shape = (64,64,1)

def build_generator():
    # noise + label input
    noise = layers.Input(shape=(latent_dim,))
    label = layers.Input(shape=(1,), dtype="int32")
    # embed & upsample label
    lbl = layers.Embedding(num_classes, latent_dim)(label)
    lbl = layers.Flatten()(lbl)
    x = layers.multiply([noise, lbl])
    # project & reshape
    x = layers.Dense(8*8*128, activation="relu")(x)
    x = layers.Reshape((8,8,128))(x)
    # upsample to 64×64
    for filters in [128,64,32]:
        x = layers.UpSampling2D()(x)
        x = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
    img = layers.Conv2D(1, 3, padding="same", activation="tanh")(x)
    return tf.keras.Model([noise,label], img, name="generator")

def build_discriminator():
    img = layers.Input(shape=img_shape)
    label = layers.Input(shape=(1,), dtype="int32")
    # embed & tile label to image shape
    lbl = layers.Embedding(num_classes, np.prod(img_shape))(label)
    lbl = layers.Reshape(img_shape)(lbl)
    x = layers.concatenate([img, lbl])
    # conv stack
    for filters in [32,64,128]:
        x = layers.Conv2D(filters, 3, strides=2, padding="same")(x)
        x = layers.LeakyReLU(0.2)(x)
    x = layers.Flatten()(x)
    validity = layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model([img,label], validity, name="discriminator")

# instantiate
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer="adam", loss="binary_crossentropy")

# combined GAN
noise_in = layers.Input(shape=(latent_dim,))
label_in = layers.Input(shape=(1,), dtype="int32")
fake_img = generator([noise_in, label_in])
discriminator.trainable = False
validity = discriminator([fake_img, label_in])
gan = tf.keras.Model([noise_in,label_in], validity)
gan.compile(optimizer="adam", loss="binary_crossentropy")


# ————————————————————————————————
# 1B) Training Loop (skeleton)
# ————————————————————————————————

def load_real_data(data_dir):
    X, y = [], []
    for cls in ["normal","reversal","corrected"]:
        label = {"normal":0,"reversal":1,"corrected":2}[cls]
        folder = os.path.join(data_dir, cls)
        for fname in os.listdir(folder):
            img = load_img(os.path.join(folder,fname), target_size=(64,64), color_mode="grayscale")
            X.append((img_to_array(img)/127.5-1.0))
            y.append(label)
    return np.array(X), np.array(y)

real_imgs, real_labels = load_real_data(r"D:\STUDY\Sumit\lectures sumit\college\sem 6\dyslexia\Dataset Dyslexia_Password WanAsy321\Gambo\Train")
batch_size = 32
steps = 2000

for step in range(steps):
    # 1) Train Discriminator
    idx = np.random.randint(0, real_imgs.shape[0], batch_size)
    imgs, lbls = real_imgs[idx], real_labels[idx]
    noise = np.random.normal(size=(batch_size, latent_dim))
    fake_lbls = lbls  # condition with same labels
    fake_imgs = generator.predict([noise, fake_lbls])
    d_loss_real = discriminator.train_on_batch([imgs,lbls], np.ones((batch_size,1)))
    d_loss_fake = discriminator.train_on_batch([fake_imgs,fake_lbls], np.zeros((batch_size,1)))
    # 2) Train Generator via gan
    g_loss = gan.train_on_batch([noise, lbls], np.ones((batch_size,1)))
    if step % 200 == 0:
        print(f"[{step}] D_real:{d_loss_real:.3f} D_fake:{d_loss_fake:.3f} G:{g_loss:.3f}")

# ————————————————————————————————
# 1C) Sample & Save New Data
# ————————————————————————————————




1/1 [==============================] - 1s 621ms/step
[0] D_real:0.754 D_fake:0.702 G:0.688
1/1 [==============================] - 0s 158ms/step


1/1 [==============================] - 0s 148ms/step
[200] D_real:0.000 D_fake:0.000 G:9.133
1/1 [==============================] - 0s 163ms/step


1/1 [==============================] - 0s 157ms/step
[400] D_real:0.004 D_fake:0.064 G:4.140
1/1 [==============================] - 0s 153ms/step


1/1 [==============================] - 0s 179ms/step
[600] D_real:0.000 D_fake:0.001 G:7.453
1/1 [==============================] - 0s 168ms/step


1/1 [==============================] - 0s 192ms/step


...

1/1 [==============================] - 0s 368ms/step
[1600] D_real:0.003 D_fake:0.001 G:6.665
1/1 [==============================] - 0s 354ms/step


1/1 [==============================] - 0s 404ms/step
[1800] D_real:0.000 D_fake:0.001 G:6.933
1/1 [==============================] - 1s 524ms/step


1/1 [==============================] - 0s 468ms/step


16/16 [==============================] - 7s 427ms/step


NameError: name 'cv2' is not defined

In [5]:
import cv2
import os

classes = ["normal", "reversal", "corrected"]
for cls in classes:
    os.makedirs(f"augmented/{cls}", exist_ok=True)

def generate_augmented_samples(n_per_class=500):
    os.makedirs("augmented", exist_ok=True)
    for cls, label in [("normal",0),("reversal",1),("corrected",2)]:
        os.makedirs(f"augmented/{cls}", exist_ok=True)
        noise = np.random.normal(size=(n_per_class, latent_dim))
        labels = np.full((n_per_class,1), label)
        gen_imgs = 0.5*(generator.predict([noise,labels]) + 1.0)  # back to [0,1]
        for i,img in enumerate(gen_imgs):
            cv2.imwrite(f"augmented/{cls}/{cls}_{i}.png", img*255)

generate_augmented_samples()

16/16 [==============================] - 5s 325ms/step


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Combine and load data using ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    "augmented",  # or a folder with both real + augmented
    target_size=(64, 64),
    color_mode="grayscale",
    class_mode="categorical",
    subset="training",
    batch_size=32
)

val_gen = datagen.flow_from_directory(
    "augmented",
    target_size=(64, 64),
    color_mode="grayscale",
    class_mode="categorical",
    subset="validation",
    batch_size=32
)

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=10)


Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Epoch 1/10
38/38 [==============================] - 13s 308ms/step - loss: 1.0987 - accuracy: 0.3250 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 2/10
38/38 [==============================] - 11s 282ms/step - loss: 1.0987 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 3/10
38/38 [==============================] - 13s 327ms/step - loss: 1.0987 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/10
38/38 [==============================] - 11s 292ms/step - loss: 1.0987 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/10
13/38 [=========>....................] - ETA: 6s - loss: 1.0985 - accuracy: 0.3606

KeyboardInterrupt: 

In [7]:
generate_augmented_samples()


16/16 [==============================] - 14s 930ms/step
